# API KEY

In [1]:
dashscope_api_key = "sk-b69887d9a9244554892e7589af2f187c"

In [12]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=dashscope_api_key,  # 如果您没有配置环境变量，请用阿里云百炼API Key将本行替换为：api_key="sk-xxx"
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope SDK的base_url
    )
    completion = client.chat.completions.create(
        model="qwen2.5-7b-instruct",  # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=[{'role': 'system', 'content': 'Please reason step by step, and put your final answer within \\boxed{{}}.'},
                  {'role': 'user', 'content': '1+1=?'}],
        )
    print(completion.model_dump_json())

if __name__ == '__main__':
    get_response()

BadRequestError: Error code: 400 - {'error': {'code': 'integer_below_min_value', 'param': 'n', 'message': '<400> InternalError.Algo.InvalidParameter: Range of n should be [1, 4]', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-c92fdf56-c06d-9d25-98bb-460cdd401842', 'request_id': 'c92fdf56-c06d-9d25-98bb-460cdd401842'}

# prepare input data

{"custom_id":"2","method":"POST","url":"/v1/chat/completions","body":{"model":"qwen2.5-7b-instruct","messages":[{"role":"system","content":"Please reason step by step, and put your final answer within \\boxed{{}}."},{"role":"user","content":problem}]}}

In [2]:
import json

input_path = "math500.jsonl"
output_path = "test.jsonl"
system_prompt = "Please reason step by step, and put your final answer within \\boxed{{}}."

with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for idx, line in enumerate(fin):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"第{idx+1}行解析失败: {e}")
            continue
        problem = obj.get("problem", "")
        out_obj = {
            "custom_id": str(idx + 1),
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "qwen2.5-7b-instruct",
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": problem}
                ]
            }
        }
        fout.write(json.dumps(out_obj, ensure_ascii=False) + "\n")
print(f"已生成 {output_path}")


已生成 test.jsonl


# Upload file

In [3]:
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，可用阿里云百炼API Key将下行替换为：api_key="sk-xxx"。但不建议在生产环境中直接将API Key硬编码到代码中，以减少API Key泄露风险。
    api_key=dashscope_api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 阿里云百炼服务的base_url
)

# test.jsonl 是一个本地示例文件，purpose必须是batch
file_object = client.files.create(file=Path("test.jsonl"), purpose="batch")

print(file_object.model_dump_json())

{"id":"file-batch-89cbe2460f1d4037bbb5e441","bytes":234290,"created_at":1753614009,"filename":"test.jsonl","object":"file","purpose":"batch","status":"processed","status_details":null}


# Create batch task

In [8]:
batch = client.batches.create(
    input_file_id="file-batch-89cbe2460f1d4037bbb5e441",  # 上传文件返回的id或OSS文件URL或OSS文件资源标识符
    endpoint="/v1/chat/completions",  # Embedding文本向量模型填写/v1/embeddings,测试模型batch-test-model填写/v1/chat/ds-test,其他模型填写/v1/chat/completions
    completion_window="24h",
    metadata={'ds_name':"math500-qwen7b",'ds_description':'任务描述'} # metadata数据，非必填字段，用于创建任务名称、描述
)
print(batch)

Batch(id='batch_90e6f1e9-a5e4-451d-ba1e-25c6aaa768d0', completion_window='24h', created_at=1753614176, endpoint='/v1/chat/completions', input_file_id='file-batch-89cbe2460f1d4037bbb5e441', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=None, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'ds_name': 'math500-qwen7b', 'ds_description': '任务描述'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


# Manage batch task

In [9]:
batch = client.batches.retrieve("batch_90e6f1e9-a5e4-451d-ba1e-25c6aaa768d0")  # 将batch_id替换为Batch任务的id
print(batch)

Batch(id='batch_90e6f1e9-a5e4-451d-ba1e-25c6aaa768d0', completion_window='24h', created_at=1753614176, endpoint='/v1/chat/completions', input_file_id='file-batch-89cbe2460f1d4037bbb5e441', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='model_not_found', line=1, message="The provided model 'qwen2.5-7b-instruct' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=2, message="The provided model 'qwen2.5-7b-instruct' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=3, message="The provided model 'qwen2.5-7b-instruct' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=4, message="The provided model 'qwen2.5-7b-instruct' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=5, message="The provided model 'qwen

# test winogrande

In [13]:
from datasets import load_dataset

# 加载Winogrande数据集
winogrande_dataset = load_dataset("winogrande", "winogrande_xs")
print(winogrande_dataset)


README.md: 0.00B [00:00, ?B/s]

winogrande_xs/train-00000-of-00001.parqu(…):   0%|          | 0.00/11.7k [00:00<?, ?B/s]

winogrande_xs/test-00000-of-00001.parque(…):   0%|          | 0.00/118k [00:00<?, ?B/s]

winogrande_xs/validation-00000-of-00001.(…):   0%|          | 0.00/85.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 160
    })
    test: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1767
    })
    validation: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1267
    })
})


In [14]:
train_dataset = winogrande_dataset["train"]
print("train split 样本数：", len(train_dataset))
print("前5个样本：")
for i in range(5):
    print(train_dataset[i])
    messages = [{'role': 'system', 'content': 'Please reason step by step, and put your final answer within \\boxed{{}}.'},
                {'role': 'user', 'content': train_dataset[i]['sentence']}]
    print(messages)


train split 样本数： 160
前5个样本：
{'sentence': "Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.", 'option1': 'Ian', 'option2': 'Dennis', 'answer': '2'}
{'sentence': "Ian volunteered to eat Dennis's menudo after already having a bowl because _ enjoyed eating intestine.", 'option1': 'Ian', 'option2': 'Dennis', 'answer': '1'}
{'sentence': 'He never comes to my home, but I always go to his house because the _ is smaller.', 'option1': 'home', 'option2': 'house', 'answer': '1'}
{'sentence': 'He never comes to my home, but I always go to his house because the _ is bigger.', 'option1': 'home', 'option2': 'house', 'answer': '2'}
{'sentence': "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate.", 'option1': 'Kyle', 'option2': 'Logan', 'answer': '2'}


In [15]:
import json

def process_sample(item: dict) -> list:
    """
    Processes a data sample to create a question-answering prompt.

    The function takes a dictionary representing a WinoGrande-style problem
    and formats it into a list of messages suitable for a conversational AI.

    Args:
        item: A dictionary with keys 'sentence', 'option1', 'option2'.
              Example: {'sentence': "The trophy doesn't fit in the suitcase because _ is too big.",
                        'option1': 'trophy', 'option2': 'suitcase', 'answer': '1'}

    Returns:
        A list of dictionaries for the prompt, containing a system and a user message.
    """

    system_prompt = """You are an expert assistant in natural language understanding and commonsense reasoning. Please reason step-by-step and answer with the option number.
"""
    
    user_prompt = f"""Complete the following sentence by choosing the most logical option to fill in the blank.

Sentence: "{item['sentence']}"

Options:
1. {item['option1']}
2. {item['option2']}

Which option is the correct choice?
"""

    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]
    
    return messages


In [16]:
messages = process_sample(train_dataset[0])
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=dashscope_api_key,  # 如果您没有配置环境变量，请用阿里云百炼API Key将本行替换为：api_key="sk-xxx"
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope SDK的base_url
    )
    completion = client.chat.completions.create(
        model="qwen2.5-7b-instruct",  # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=messages,
        )
    print(completion.model_dump_json())

if __name__ == '__main__':
    get_response()

{"id":"chatcmpl-545b36a5-a1d9-984d-97e8-f79af0f805a1","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Let's reason through this step-by-step:\n\n1. The sentence states that Ian volunteered to eat Dennis's menudo, which is an unusual dish containing intestines.\n2. It also mentions that Ian had already had a bowl of it.\n3. The sentence ends with \"because _ despised eating intestine. \"\n4. We need to determine who despises eating intestines - Ian or Dennis.\n\nGiven these points, we can conclude:\n- Ian has already eaten a bowl of menudo, which contains intestines.\n- Despite having already eaten some, Ian is still willing to eat more menudo.\n- This implies that Ian does not despise eating intestines, as he's not refusing to eat them again.\n\nTherefore, the blank should be filled with Dennis, as he is the one who would likely despise eating intestine due to his own menudo.\n\nThe most logical option is:\nOption 2: Dennis","refusal":null,"role":"ass